In [2]:
# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os


/Users/bhawnayadav/Desktop/mlops/newenv/lib/python3.12/site-packages/mlflow/utils/requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [3]:
import dagshub


mlflow.set_tracking_uri("https://dagshub.com/yadavdipu296/mlops.mlflow")
dagshub.init(repo_owner='yadavdipu296', repo_name='mlops', mlflow=True)

Accessing as yadavdipu296

Initialized MLflow to track repo "yadavdipu296/mlops"

Repository yadavdipu296/mlops initialized!

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [5]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise


<>:30: SyntaxWarning: invalid escape sequence '\s'
<>:30: SyntaxWarning: invalid escape sequence '\s'
/var/folders/g_/sy0vrt390g59cw_vcq84l2br0000gn/T/ipykernel_1134/720468952.py:30: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [6]:
df=normalize_text(df)

In [7]:
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [8]:

x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [9]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


/var/folders/g_/sy0vrt390g59cw_vcq84l2br0000gn/T/ipykernel_1134/468518138.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


In [10]:
mlflow.set_experiment("bow_vs_tfidf")

vectorizers = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

algorithms = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

In [15]:
with mlflow.start_run(run_name="All Experiments") as parent_run:
    # Assuming `algorithms` is a dict like: {'logistic_regression': LogisticRegression(), ...}
    # and `vectorizers` is a dict like: {'tfidf': TfidfVectorizer(), ...}
    for algo_name, algo_obj in algorithms.items():
        for vec_name, vec_obj in vectorizers.items():
            # Use the clear, short string names for the run_name
            run_name = f"{algo_name}_{vec_name}"
            with mlflow.start_run(run_name=run_name, nested=True) as child_run:
                # Use the objects for the machine learning operations
                x = vec_obj.fit_transform(df['content'])
                y = df['sentiment']

                # Log the data in MLflow
                X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

                # Log parameters
                mlflow.log_params({
                    'Vectorizer': vec_name,
                    'test_size': 0.2,
                    'Model': algo_name
                })

                model = algo_obj
                model.fit(X_train, y_train)

                # Log the model
                mlflow.sklearn.log_model(model, "model")

                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)

                # Define a dictionary to map algorithms to their parameters
                algorithm_params = {
                    'LogisticRegression': ['C'],
                    'MultinomialNB': ['alpha'],
                    'XGBoost': ['n_estimators', 'learning_rate'],
                    'RandomForest': ['n_estimators', 'max_depth'],
                    'GradientBoosting': ['n_estimators', 'learning_rate', 'max_depth']
                }

                # Log model parameters
                for param in algorithm_params.get(algo_obj.__class__.__name__, []):
                    mlflow.log_param(param, getattr(model, param))

                # Log metrics
                mlflow.log_metrics({
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1
                })

                print(f"Algorithm: {algo_name}, Feature Engineering: {vec_name}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1}")


Algorithm: Logistic Regression, Feature Engineering: BoW
Accuracy: 0.7937349397590362
Precision: 0.7846750727449079
Recall: 0.7970443349753694
F1 Score: 0.7908113391984359
Algorithm: Logistic Regression, Feature Engineering: TF-IDF
Accuracy: 0.7942168674698795
Precision: 0.777882797731569
Recall: 0.8108374384236453
F1 Score: 0.79401833092137
Algorithm: Naive Bayes, Feature Engineering: BoW
Accuracy: 0.7826506024096386
Precision: 0.7797619047619048
Recall: 0.774384236453202
F1 Score: 0.7770637666831438
Algorithm: Naive Bayes, Feature Engineering: TF-IDF
Accuracy: 0.7826506024096386
Precision: 0.7737864077669903
Recall: 0.7852216748768472
F1 Score: 0.7794621026894866
Algorithm: XGBoost, Feature Engineering: BoW
Accuracy: 0.771566265060241
Precision: 0.7988950276243094
Recall: 0.7123152709359606
F1 Score: 0.753125
Algorithm: XGBoost, Feature Engineering: TF-IDF
Accuracy: 0.7614457831325301
Precision: 0.7170283806343907
Recall: 0.8463054187192118
F1 Score: 0.7763217352010845
Algorithm: Ran